In [1]:
!pip install pymystem3
!pip install nltk

In [2]:
# Импортируем библиотеки
from pymystem3 import Mystem
import re

# Создаем анализатор для приведения слов к начальной форме
mystem = Mystem()

def get_text_features(text):
    """
    Извлекает простые признаки из текста
    Возвращает словарь с 4 признаками
    """
    # 1. Очищаем текст
    text = text.lower()
    text = text.replace('ё', 'е')
    text = re.sub(r'[^а-яё\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # 2. Делим на слова
    words = text.split()

    if not words:  # Если текст пустой
        return {
            'avg_word_len': 0,
            'unique_ratio': 0,
            'long_words_ratio': 0,
            'stopwords_ratio': 0
        }

    # 3. Считаем признаки

    # Признак 1: Средняя длина слова
    avg_word_len = sum(len(word) for word in words) / len(words)

    # Признак 2: Процент уникальных слов
    unique_ratio = len(set(words)) / len(words)

    # Признак 3: Процент длинных слов (больше 6 букв)
    long_words = [word for word in words if len(word) > 6]
    long_words_ratio = len(long_words) / len(words)

    # Признак 4: Процент стоп-слов (самых частых русских слов)
    common_words = {'и', 'в', 'не', 'на', 'я', 'быть', 'с', 'что', 'а', 'по', 'это', 'он', 'к', 'но', 'они', 'мы', 'как', 'у', 'вы', 'от', 'за', 'же', 'из', 'ли', 'то', 'все', 'о', 'при', 'так', 'же'}
    stopwords_count = sum(1 for word in words if word in common_words)
    stopwords_ratio = stopwords_count / len(words)

    # Возвращаем признаки
    return {
        'avg_word_len': avg_word_len,
        'unique_ratio': unique_ratio,
        'long_words_ratio': long_words_ratio,
        'stopwords_ratio': stopwords_ratio
    }


def train_genre_classifier():
    """
    Создает 'мозг' алгоритма - запоминает признаки разных жанров
    Возвращает 'базу знаний' о жанрах
    """
    # Это наши "учебные материалы" - примеры текстов каждого жанра
    training_texts = {
        'художественный': [
            "Лейтенант шел по желтому строительному песку, нагретому дневным палящим солнцем. Он был мокрым от кончиков пальцев до кончиков волос, все его тело было усеяно царапинами от острой колючей проволоки и ныло от сводящей с ума боли, но он был жив и направлялся к командному штабу, который виднелся на горизонте метрах в пятистах." "Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему. Всё смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это продолжалось уже третий день и мучительно чувствовалось и самими супругами, и всеми членами семьи, и домочадцами. Все члены семьи и домочадцы чувствовали, что нет смысла в их сожительстве и что на каждом постоялом дворе случайно сошедшиеся люди более связаны между собой, чем они, члены семьи и домочадцы Облонских. Жена не выходила из своих комнат, мужа третий день не было дома. Дети бегали по всему дому, как потерянные; англичанка поссорилась с экономкой и написала записку приятельнице, прося приискать ей новое место; повар ушел еще вчера со двора, во время обеда; черная кухарка и кучер просили расчета."
        "Когда мистер Бильбо Бэггинс из Бэг-Энда объявил, что вскоре отпразднует свое стоодиннадцатилетие особым пышным пиром, по всему Хоббитану поползли слухи и пересуды. Хоббитан — так звалась самая что ни на есть добротная и ухоженная хоббичья страна во всем мире — был родиной и излюбленным местом обитания этой незаметной, но весьма древней народности. В наши дни хоббиты стали, пожалуй, еще незаметнее, чем в старину: они избегают Высокого Народа, как зовут они нас, людей, и обладают чудом проворства и способностью бесшумно исчезать, едва заслышат непрошеного гостя. Уши у них чуткие, а глаза зоркие; хоть они и склонны к полноте и не любят спешки, но движутся легко и ловко."
        "Много лет спустя, пред расстрелом, полковник Аурелиано Буэндиа вспомнил тот далекий послеполуденный час, когда отец взял его с собой посмотреть на лед. Макондо было тогда небольшим селением с двумя десятками хижин, выстроенных из глины и бамбука на берегу реки, которая мчала свои прозрачные воды по ложу из белых, отполированных, как доисторические яйца, валунов. Мир был так нов, что многие вещи не имели названия и на них приходилось показывать пальцем."
        ],
        'научный': [
            "Исходя из результатов эксперимента, можно сделать заключение, что объект имеет мягкую однородную структуру, свободно пропускает свет и может изменять ряд своих параметров при воздействии на него разности потенциалов в диапазоне от 5 до 33 000 В. Исследования также показали, что объект необратимо изменяет свою молекулярную структуру под воздействием температуры свыше 300 К. При механическом воздействии на объект с силой до 1000 Н видимых изменений в структуре не наблюдается."
        "Второй закон Ньютона устанавливает количественную связь между силой, действующей на тело, и приобретаемым им ускорением. В инерциальной системе отсчета ускорение тела прямо пропорционально равнодействующей всех приложенных к нему сил и обратно пропорционально его массе: F = ma, (1) где F — вектор равнодействующей силы (Н), m — масса тела (кг), a — вектор ускорения (м/с²). Данное соотношение справедливо для тел, скорость которых значительно меньше скорости света. Для релятивистских скоростей необходимо применять законы специальной теории относительности, где зависимость между силой и ускорением становится нелинейной."
        "Принцип работы газотурбинного двигателя (ГТД) основан на преобразовании энергии сгорающего топлива в механическую работу на валу турбины. Воздух, забираемый воздухозаборником (1), поступает в многоступенчатый компрессор (2), где сжимается до высокого давления. Затем сжатый воздух направляется в камеру сгорания (3), куда через форсунки впрыскивается топливо. Образовавшаяся топливно-воздушная смесь воспламеняется, и продукты сгорания, обладающие высокой температурой и давлением, расширяются, воздействуя на лопатки газовой турбины (4). Турбина приводит во вращение как компрессор, так и полезную нагрузку (генератор, винт) через выходной вал (5). КПД современного ГТД достигает 35-40% и зависит, в первую очередь, от степени повышения давления в компрессоре и температуры газов перед турбиной."

        ],
        'публицистический': [
            "Невероятное открытие! Житель глухой деревни Эксперименталово изобрел новый препарат, заставляющий куриц нести золотые яйца! Тайна, над которой не одно столетие бились величайшие алхимики мира, наконец, раскрыта нашим соотечественником! Пока от изобретателя никаких комментариев не поступало, он, в данный момент, находится в сильном запое, однако можно однозначно сказать, что открытия таких патриотов, однозначно, стабилизируют экономику нашей страны и укрепят ее позиции на мировой арене как лидера в области добычи золота и производстве золотых изделий на десятки лет вперед."
        "ХЕЛЬСИНКИ, 28 января. Финские астрономы-любители и профессиональные исследователи из Университета Хельсинки впервые зафиксировали и описали ранее неизвестный тип полярного сияния, напоминающий по структуре рябь песчаных дюн. Явление было обнаружено на снимках, сделанных в одной из северных провинций Финляндии. По мнению ученых, «дюны» возникают в редком слое атмосферы на высоте около 100 километров и могут пролить свет на процессы, происходящие на границе с космосом."
        "Цифровое неравенство: новая граница социального расслоения Пока мы восхищаемся умными домами и обсуждаем этику беспилотных автомобилей, почти 40% населения планеты по-прежнему не имеют регулярного доступа к интернету. Это не просто статистика — это новый, глубочайший разрыв, который определяет жизненные шансы человека в XXI веке. Цифровое неравенство перестало быть вопросом лишь наличия гаджета. Речь теперь о качестве подключения, цифровой грамотности, доступе к образовательным платформам и телемедицине. Пандемия COVID-19 обнажила эту проблему как никогда: пока одни школьники учились на интерактивных вебинарах, другие выпали из образовательного процесса на месяцы из-за отсутствия стабильного интернета. Государства, которые не вложат ресурсы в ликвидацию этой «цифровой пропасти» сегодня, завтра получат потерянное поколение и замедление экономического роста. Инвестиции в инфраструктуру — это не технический вопрос, а вопрос социальной справедливости и национальной безопасности."
        ],
        'разговорный': [
            "Вы когда нибудь задумывалить о том, что было бы, если бы Земля поменялась местами с Юпитером? Я серьезно! Возникли бы Новые Васюки на его кольцах? Конечно нет! Они же из газа! Неужели вы хоть на минуту купились на такую откровенную чушь? В жизни не поверю! А если бы луна упала в Тихий Океан, на сколько бы поднялся его уровень? Вы, наверное, думаете, что я – редкий зануда, но, если я не задам эти вопросы, то кто?"
        "Где же ключи? Вечно они куда-то деваются! Так, спокойно. В сумке нет. В кармане вчерашней куртки... О, вот они, слава богу. Черт, уже без десяти девять! Автобус, наверное, прозевал. Таксовать что ли? Дорого... Эх, опять нарежут за опоздание. Ладно, побежал, авось успею."
        "Так… Что тут у нас… Картошка есть, лук… А морковка-то вся! Совсем забыла купить. Ну ничего, обойдёмся. Ой, а помидоры-то уже мягкие, надо бы быстрее использовать. Ладно, ща сооргусь. Где нож-то острый? Вот этот как будто тупой-тупой… Ага, этот лучше. Так, лук режу… Ой, глаза щиплет! Надо было под холодную воду подержать. Ну вот, уже легче. Теперь картошку… Ой, сковородка греется? Чуть не пригорело масло! Всё, теперь всё кидаю… Шкворчит, хорошо. Соль, перец… Чем бы ещё посыпать? Паприку бы… Нашлась! Идеально."
        ]
    }

    # Собираем "базу знаний" о каждом жанре
    genre_knowledge = {}

    for genre, texts in training_texts.items():
        # Собираем все признаки для всех текстов этого жанра
        all_features = []
        for text in texts:
            features = get_text_features(text)
            all_features.append(features)

        # Находим СРЕДНИЕ значения признаков для этого жанра
        avg_features = {
            'avg_word_len': sum(f['avg_word_len'] for f in all_features) / len(all_features),
            'unique_ratio': sum(f['unique_ratio'] for f in all_features) / len(all_features),
            'long_words_ratio': sum(f['long_words_ratio'] for f in all_features) / len(all_features),
            'stopwords_ratio': sum(f['stopwords_ratio'] for f in all_features) / len(all_features)
        }

        genre_knowledge[genre] = avg_features

    return genre_knowledge


def predict_genre(text, genre_knowledge):
    """
    Определяет жанр нового текста
    Сравнивает его признаки с 'базой знаний' о жанрах
    """
    # 1. Извлекаем признаки из нового текста
    text_features = get_text_features(text)

    # 2. Сравниваем с каждым жанром из базы знаний
    distances = {}

    for genre, genre_features in genre_knowledge.items():
        # Считаем "расстояние" между текстом и жанром
        # Чем меньше расстояние - тем больше похож

        distance = 0
        # Сравниваем каждый признак
        for key in text_features.keys():
            # Разница по каждому признаку
            diff = abs(text_features[key] - genre_features[key])
            distance += diff

        distances[genre] = distance

    # 3. Находим самый похожий жанр (с наименьшим расстоянием)
    predicted_genre = min(distances, key=distances.get)

    # 4. Возвращаем результат
    return predicted_genre, distances


def main():
    """
    Главная функция - запускает весь алгоритм
    """
    print("=" * 60)
    print("АЛГОРИТМ ОПРЕДЕЛЕНИЯ ЖАНРА ТЕКСТА")
    print("=" * 60)

    # 1. Обучаем алгоритм (создаем базу знаний)
    print("\n1. Обучаю алгоритм на примерах...")
    genre_knowledge = train_genre_classifier()

    print("База знаний создана! Знаю 4 жанра:")
    for genre in genre_knowledge:
        print(f"  - {genre}")

    # 2. Получаем текст от пользователя
    print("\n2. Введите текст для анализа:")
    print("(Можно вставить любой русский текст)")
    print("-" * 40)
    # Пример текста для тестирования (пользователь заменит на свой)
    user_text = """Пьер так и не успел выбрать себе карьеры в Петербурге и действительно был выслан в Москву за буйство. История, которую рассказывали у графа Ростова, была справедлива. Пьер участвовал в связыванье квартального с медведем. Он приехал несколько дней тому назад и остановился, как всегда, в доме своего отца. Хотя он и предполагал, что история его уже известна в Москве и что дамы, окружающие его отца, всегда недоброжелательные к нему, воспользуются этим случаем, чтобы раздражить графа, он все-таки в день приезда пошел на половину отца. Войдя в гостиную, обычное местопребывание княжон, он поздоровался с дамами, сидевшими за пяльцами и за книгой, которую вслух читала одна из них. Их было три."""

    print(f"Текст для анализа:\n{user_text}")

    # 3. Определяем жанр
    print("\n3. Анализирую текст...")
    predicted_genre, distances = predict_genre(user_text, genre_knowledge)

    # 4. Показываем результат
    print("\n4. РЕЗУЛЬТАТ АНАЛИЗА:")
    print("=" * 40)
    print(f"Определенный жанр: {predicted_genre.upper()}")
    print("\nНасколько текст похож на каждый жанр:")
    print("(чем меньше число - тем больше похож)")
    print("-" * 40)

    for genre, distance in sorted(distances.items(), key=lambda x: x[1]):
        similarity = max(0, 100 - distance * 20)  # Преобразуем в проценты
        print(f"{genre:20} - {distance:.3f} ({similarity:.0f}% похоже)")

    print("=" * 60)


# Запускаем программу
if __name__ == "__main__":
    main()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


АЛГОРИТМ ОПРЕДЕЛЕНИЯ ЖАНРА ТЕКСТА

1. Обучаю алгоритм на примерах...
База знаний создана! Знаю 4 жанра:
  - художественный
  - научный
  - публицистический
  - разговорный

2. Введите текст для анализа:
(Можно вставить любой русский текст)
----------------------------------------
Текст для анализа:
Пьер так и не успел выбрать себе карьеры в Петербурге и действительно был выслан в Москву за буйство. История, которую рассказывали у графа Ростова, была справедлива. Пьер участвовал в связыванье квартального с медведем. Он приехал несколько дней тому назад и остановился, как всегда, в доме своего отца. Хотя он и предполагал, что история его уже известна в Москве и что дамы, окружающие его отца, всегда недоброжелательные к нему, воспользуются этим случаем, чтобы раздражить графа, он все-таки в день приезда пошел на половину отца. Войдя в гостиную, обычное местопребывание княжон, он поздоровался с дамами, сидевшими за пяльцами и за книгой, которую вслух читала одна из них. Их было три.

3. Ан